Import libraries

In [1]:
from pyspark.sql import SparkSession

In [17]:
from pyspark.ml.recommendation import ALS
from pyspark.ml.evaluation import RegressionEvaluator
import findspark
findspark.init()
import pandas as pd

ModuleNotFoundError: No module named 'findspark'

Import SparkSession from pyspark.sql

Create an instance of SparkSession

In [3]:
spark = SparkSession.builder.appName('recommender').getOrCreate()

Print the tables in the catalog

In [4]:
print(spark.catalog.listTables())

[]


Load the file

In [5]:
file_path = "/home/alainkuiete/Documents/DATA612/rating.csv"

Read the rating data

In [6]:
ratings = spark.read.csv(file_path, header=True)

Show the ratings data

In [7]:
ratings.show()

+------+-------+------+-------------------+
|userId|movieId|rating|          timestamp|
+------+-------+------+-------------------+
|     1|      2|   3.5|2005-04-02 23:53:47|
|     1|     29|   3.5|2005-04-02 23:31:16|
|     1|     32|   3.5|2005-04-02 23:33:39|
|     1|     47|   3.5|2005-04-02 23:32:07|
|     1|     50|   3.5|2005-04-02 23:29:40|
|     1|    112|   3.5|2004-09-10 03:09:00|
|     1|    151|     4|2004-09-10 03:08:54|
|     1|    223|     4|2005-04-02 23:46:13|
|     1|    253|     4|2005-04-02 23:35:40|
|     1|    260|     4|2005-04-02 23:33:46|
|     1|    293|     4|2005-04-02 23:31:43|
|     1|    296|     4|2005-04-02 23:32:47|
|     1|    318|     4|2005-04-02 23:33:18|
|     1|    337|   3.5|2004-09-10 03:08:29|
|     1|    367|   3.5|2005-04-02 23:53:00|
|     1|    541|     4|2005-04-02 23:30:03|
|     1|    589|   3.5|2005-04-02 23:45:57|
|     1|    593|   3.5|2005-04-02 23:31:01|
|     1|    653|     3|2004-09-10 03:08:11|
|     1|    919|   3.5|2004-09-1

Add ratings to catalog

In [8]:
ratings.createOrReplaceTempView("ratings")

Look at the type of each column

In [9]:
ratings.printSchema()

root
 |-- userId: string (nullable = true)
 |-- movieId: string (nullable = true)
 |-- rating: string (nullable = true)
 |-- timestamp: string (nullable = true)



Cast the columns to integers

In [10]:
ratings = ratings.withColumn("userId", ratings.userId.cast("integer"))
ratings = ratings.withColumn("movieId", ratings.movieId.cast("integer"))
ratings = ratings.withColumn("rating", ratings.rating.cast("float"))

In [11]:
ratings.printSchema()

root
 |-- userId: integer (nullable = true)
 |-- movieId: integer (nullable = true)
 |-- rating: float (nullable = true)
 |-- timestamp: string (nullable = true)



Eliminate the timestamp column in ratings dataframe

In [12]:
ratings = ratings.select(['userId', 'movieId', 'rating'])

Summirized Statistics on the data ratings

In [13]:
ratings.describe().show()

+-------+-----------------+------------------+------------------+
|summary|           userId|           movieId|            rating|
+-------+-----------------+------------------+------------------+
|  count|         20000263|          20000263|          20000263|
|   mean|69045.87258292554| 9041.567330339605|3.5255285642993797|
| stddev|40038.62665316145|19789.477445413166|1.0519889192942444|
|    min|                1|                 1|               0.5|
|    max|           138493|            131262|               5.0|
+-------+-----------------+------------------+------------------+



Splitting the data into train and test sets

In [14]:
training, test = ratings.randomSplit([0.8,0.2])

In [16]:
als = ALS(maxIter=5, regParam=0.09, userCol='userId', itemCol='movieId', ratingCol='rating')

model = als.fit(training)

predictions = model.transform(test)

ERROR:root:Exception while sending command.
Traceback (most recent call last):
  File "/home/alainkuiete/anaconda3/lib/python3.7/site-packages/py4j/java_gateway.py", line 1159, in send_command
    raise Py4JNetworkError("Answer from Java side is empty")
py4j.protocol.Py4JNetworkError: Answer from Java side is empty

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/home/alainkuiete/anaconda3/lib/python3.7/site-packages/py4j/java_gateway.py", line 985, in send_command
    response = connection.send_command(command)
  File "/home/alainkuiete/anaconda3/lib/python3.7/site-packages/py4j/java_gateway.py", line 1164, in send_command
    "Error while receiving", e, proto.ERROR_ON_RECEIVE)
py4j.protocol.Py4JNetworkError: Error while receiving


Py4JError: org does not exist in the JVM

Summary statistic of predictions


In [ ]:
predictions.describe().show()

Create a mapping for movie ids

In [ ]:
predictions = predictions.na.drop()
predictions.describe().show()

In [ ]:

Evaluation

In [ ]:
evaluator = RegressionEvaluator(metricName='rmse', labelCol='rating')
rmse = evaluator.evaluate(predictions)
rmse